In [1]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda')

(device(type='cpu'), device(type='cuda'))

查看有多少可用的GPU

In [2]:
torch.cuda.device_count() 

1

In [3]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1: 
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 张量与GPU

In [4]:
# 我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

无论何时我们要对多个项进行操作， 它们都必须在同一个设备上

In [5]:
# 在GPU上创建tensor
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [6]:
# X已经在cuda:0上了，此时调用X.cuda(0)不会复制并分配新内存
X.cuda(0) is X

True

## 神经网络与GPU

In [16]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu()) # 将模型移动到GPU上

In [17]:
# 当输入的张量在GPU上，模型将在同一GPU上计算结果，结果也在同一GPU上
net(X)

tensor([[-0.6501],
        [-0.6501]], device='cuda:0', grad_fn=<AddmmBackward>)

In [9]:
net[0].weight.data.device

device(type='cuda', index=0)

只要所有的数据和模型参数都在同一个设备上(模型net、损失函数loss、数据X，y)， 我们就可以有效地学习模型